### 安装 modelscope

In [3]:
!pip install modelscope

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### 下载模型

In [4]:
!modelscope download --model Qwen/Qwen3-0.6B --local_dir ./models/Qwen3-0.6B


 _   .-')                _ .-') _     ('-.             .-')                              _ (`-.    ('-.
( '.( OO )_             ( (  OO) )  _(  OO)           ( OO ).                           ( (OO  ) _(  OO)
 ,--.   ,--.).-'),-----. \     .'_ (,------.,--.     (_)---\_)   .-----.  .-'),-----.  _.`     \(,------.
 |   `.'   |( OO'  .-.  ',`'--..._) |  .---'|  |.-') /    _ |   '  .--./ ( OO'  .-.  '(__...--'' |  .---'
 |         |/   |  | |  ||  |  \  ' |  |    |  | OO )\  :` `.   |  |('-. /   |  | |  | |  /  | | |  |
 |  |'.'|  |\_) |  |\|  ||  |   ' |(|  '--. |  |`-' | '..`''.) /_) |OO  )\_) |  |\|  | |  |_.' |(|  '--.
 |  |   |  |  \ |  | |  ||  |   / : |  .--'(|  '---.'.-._)   \ ||  |`-'|   \ |  | |  | |  .___.' |  .--'
 |  |   |  |   `'  '-'  '|  '--'  / |  `---.|      | \       /(_'  '--'\    `'  '-'  ' |  |      |  `---.
 `--'   `--'     `-----' `-------'  `------'`------'  `-----'    `-----'      `-----'  `--'      `------'

Processing 9 items:   0%|                           |

### 使用 modelscope sdk 下载模型

In [5]:
from modelscope import snapshot_download
model_dir = snapshot_download("Qwen/Qwen3-0.6B")

2025-06-25 23:09:32,263 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-06-25 23:09:44,903 - modelscope - INFO - Download model 'Qwen/Qwen3-0.6B' successfully.
2025-06-25 23:09:44,907 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/models/Qwen/Qwen3-0.6B].


### 安装 git lfs 下载模型

In [6]:
!git lfs install

Git LFS initialized.


In [7]:
!git clone https://www.modelscope.cn/Qwen/Qwen3-Embedding-0.6B.git

正克隆到 'Qwen3-Embedding-0.6B'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 58 (delta 22), reused 39 (delta 10), pack-reused 0
接收对象中: 100% (58/58), 1.73 MiB | 610.00 KiB/s, 完成.
处理 delta 中: 100% (22/22), 完成.
过滤内容: 100% (2/2), 1.12 GiB | 25.44 MiB/s, 完成.


### 安装 transformers

In [8]:
!pip install transformers

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### 使用 AutoModel 加载模型

In [17]:
from transformers import modeling_utils

if not hasattr(modeling_utils, "ALL_PARALLEL_STYLES") or modeling_utils.ALL_PARALLEL_STYLES is None:
    modeling_utils.ALL_PARALLEL_STYLES = ["tp", "none", "colwise", "rowwise"]


In [18]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

2025-06-25 23:46:49,950 - modelscope - INFO - Target directory already exists, skipping creation.


2025-06-25 23:46:51,309 - modelscope - INFO - Target directory already exists, skipping creation.


thinking content: <think>
Okay, the user asked for a short introduction to a large language model. Let me start by recalling what I know. Large language models are AI systems that can understand and generate human language. They're trained on massive amounts of text data, so they can learn patterns and nuances.

I should mention their ability to understand and generate text, maybe give an example like answering questions. Also, they're used in various fields like healthcare, customer service, and creative writing. It's important to highlight their purpose and application without getting too technical. Keep it concise but informative.
</think>
content: A large language model (LLM) is an AI system designed to understand and generate human language. These models are trained on vast amounts of text data, allowing them to comprehend context, nuances, and complex ideas. They can answer questions, write creative content, and assist in various tasks across different domains, making them valuab

### 使用 pipeline 使用模型

In [21]:
from modelscope.pipelines import pipeline
text_generation = pipeline('text-generation',model='Qwen/Qwen3-0.6B')

[INFO:swift.hub.hub] Downloading the model from ModelScope Hub, model_id: Qwen/Qwen3-0.6B


[INFO:modelscope] Target directory already exists, skipping creation.
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/models/Qwen/Qwen3-0___6B
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift.hub.hub] Downloading the model from ModelScope Hub, model_id: Qwen/Qwen3-0.6B


[INFO:modelscope] Target directory already exists, skipping creation.
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/models/Qwen/Qwen3-0___6B
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] args.result_path: /mnt/workspace/notebooks/result/Qwen3-0.6B/infer_result/20250625-235542.jsonl
[INFO:swift] Setting args.eval_human: True
[INFO:swift.hub.hub] Downloading the model from ModelScope Hub, model_id: Qwen/Qwen3-0.6B


[INFO:modelscope] Target directory already exists, skipping creation.
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/models/Qwen/Qwen3-0___6B
[INFO:swift] model_kwargs: {'device_map': 'cpu'}
[INFO:swift] default_system: None
[INFO:swift] response_prefix: ''
[INFO:swift] agent_template: hermes
[INFO:swift] max_length: 40960
[INFO:swift] norm_bbox: norm1000
[INFO:modelscope] cuda is not available, using cpu instead.


### pipeline 是处理任务最简单的方式
modelscope 原生支持 pipeline，兼容 transformers 的 pipeline

In [22]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

inference_pipeline = pipeline(
    task=Tasks.auto_speech_recognition,
    model='iic/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch',
    model_revision="v2.0.4")

rec_result = inference_pipeline('https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_audio/asr_vad_punc_example.wav')
print(rec_result)

[INFO:modelscope] Use user-specified model revision: v2.0.4


[INFO:modelscope] Use user-specified model revision: v2.0.4
[INFO:modelscope] Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

[INFO:modelscope] Download model 'iic/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch' successfully.
[INFO:modelscope] initiate model from /mnt/workspace/.cache/modelscope/models/iic/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch
[INFO:modelscope] initiate model from location /mnt/workspace/.cache/modelscope/models/iic/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch.
[INFO:modelscope] initialize model from /mnt/workspace/.cache/modelscope/models/iic/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch


funasr version: 1.2.6.
Check update of funasr, and it would cost few times. You may disable it by set `disable_update=True` in AutoModel
You are using the latest version of funasr-1.2.6


...

[INFO:modelscope] Download model 'iic/speech_fsmn_vad_zh-cn-16k-common-pytorch' successfully.


[WARNING:modelscope] Model revision not specified, use revision: v2.0.4
[INFO:modelscope] Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

[INFO:modelscope] Download model 'iic/punc_ct-transformer_cn-en-common-vocab471067-large' successfully.
Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.685 seconds.
DEBUG:jieba:Loading model cost 0.685 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.
[WARNING:modelscope] No preprocessor field found in cfg.
[WARNING:modelscope] No val key and type key found in preprocessor domain of configuration.json file.
[WARNING:modelscope] Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/mnt/workspace/.cache/modelscope/models/iic/speech_paraformer-large-vad-punc_asr_nat-zh-cn-16k-common-vocab8404-pytorch'}. trying to build by task and model information.
[INFO:modelscope] No preprocessor key ('funasr', 'a

[{'key': 'asr_vad_punc_example', 'text': '正是因为存在绝对正义，所以我们接受现实的相对正义。但是不要因为现实的相对正义，我们就认为这个世界没有正义。因为如果当你认为这个世界没有正义。', 'timestamp': [[430, 670], [670, 810], [810, 1030], [1030, 1130], [1130, 1330], [1330, 1510], [1510, 1670], [1670, 1810], [1810, 1970], [1970, 2210], [2250, 2390], [2390, 2490], [2490, 2570], [2570, 2710], [2710, 2950], [2970, 3210], [3310, 3550], [3570, 3730], [3730, 3830], [3830, 3970], [3970, 4150], [4150, 4270], [4270, 4535], [5290, 5470], [5470, 5610], [5610, 5710], [5710, 5910], [5910, 6070], [6070, 6230], [6230, 6470], [6470, 6650], [6650, 6750], [6750, 6950], [6950, 7130], [7130, 7250], [7250, 7490], [7490, 7590], [7590, 7710], [7710, 7910], [7910, 8070], [8070, 8290], [8290, 8430], [8430, 8550], [8550, 8710], [8710, 8950], [9050, 9290], [9370, 9550], [9550, 9790], [9790, 9965], [10600, 10760], [10760, 10900], [10900, 11120], [11120, 11300], [11300, 11400], [11400, 11580], [11580, 11700], [11700, 11800], [11800, 11920], [11920, 12020], [12020, 12160], [12160, 12320]

### 模型微调
ms-swift 是魔搭社区提供的大模型与多模态大模型微调部署框架，支持非常多的模型以及汇集了很多训练技术

#### 安装

In [23]:
!pip install ms-swift -U

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.4/669.4 kB 32.9 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: ms-swift
    Found existing installation: ms-swift 3.4.1.post1
    Uninstalling ms-swift-3.4.1.post1:
      Successfully uninstalled ms-swift-3.4.1.post1

[notice] A new release of pip is available: 23.3.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


使用 swift 框架对 Qwen2.5-7B-Instruct 模型进行 LoRA 微调

In [ ]:
# 22GB
CUDA_VISIBLE_DEVICES=0 \
swift sft \
    --model Qwen/Qwen2.5-7B-Instruct \
    --train_type lora \
    --dataset 'AI-ModelScope/alpaca-gpt4-data-zh#500' \
              'AI-ModelScope/alpaca-gpt4-data-en#500' \
              'swift/self-cognition#500' \
    --torch_dtype bfloat16 \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --learning_rate 1e-4 \
    --lora_rank 8 \
    --lora_alpha 32 \
    --target_modules all-linear \
    --gradient_accumulation_steps 16 \
    --eval_steps 50 \
    --save_steps 50 \
    --save_total_limit 2 \
    --logging_steps 5 \
    --max_length 2048 \
    --output_dir output \
    --system 'You are a helpful assistant.' \
    --warmup_ratio 0.05 \
    --dataloader_num_workers 4 \
    --model_author swift \
    --model_name swift-robot

使用训练后的参数进行推理

In [ ]:
CUDA_VISIBLE_DEVICES=0 \
swift infer \
    --adapters output/vx-xxx/checkpoint-xxx \
    --stream true \
    --temperature 0 \
    --max_new_tokens 2048

### 模型部署
SwingDeploy 支持一键部署模型到云资源上，并且提供兼容 OpenAI 的 API 接口;
对于自用 GPU 环境推荐使用 vllm 部署模型

In [ ]:
!vllm serve Qwen/Qwen2.5-7B-Instruct --enable-auto-tool-choice --tool-call-parser hermes